# 2020 1C 3ra Oportunidad

(***) Un equipo del Ministerio de Salud de la Nación se encuentra realizando distintos análisis de datos a partir de la información que tienen unificada a nivel nacional de los sistemas de información para el seguimiento de la campaña de vacunación contra el sars-cov-2.

Es por ello que cuentan con información en los siguientes archivos csv:

```
centro_de_vacunacion.csv (id_centro_vacunacion, nombre, direccion, codigo_postal, partido, provincia)
```

```
categoria_plan_vacunacion.csv  (id_categoria, nombre_categoria, prioridad)
```

```
aplicaciones.csv (id_ciudadano, fecha_aplicacion, id_categoria, id_centro_vacunacion, id_unico_dosis_vacuna, id_tipo_vacuna)
```

El primer archivo cuenta con información de los distintos centros de vacunación a lo largo y ancho del país donde se realizan aplicaciones de las vacunas disponibles. Este archivo nos da información geográfica de donde se encuentran los centros de vacunación en cada provincia.

El segundo tiene información de las distintas categorías que tiene el plan de vacunación, y la prioridad de las mismas, las cuales son valores numéricos siendo el valor 1 el valor más alto de prioridad. Por ejemplo, la categoría “Personal Esencial: Salud” tiene prioridad 1. Tener en cuenta que múltiples categorías pueden tener el mismo número de prioridad.

En el último archivo, contamos con la información de ciudadanos que se fueron realizando aplicaciones dentro del plan de vacunación, cuando se aplicó la misma (en formato “YYYY-mm-dd”), en que lugar y bajo qué categoría se hizo. Algo a considerar aquí es que también contamos con información específica de la dosis que se le aplicó a la personas y el id del tipo de vacuna.

Algo importante a considerar es que en la Argentina solamente se cuenta por el momento con vacunas que requieren 2 dosis para tener inmunidad completa, es por eso que en aplicaciones.csv pueden haber múltiples entradas por ciudadano.

Se desea:

a) Indicar el porcentaje de ciudadanos vacunados en prioridad 1 que tienen inmunidad completa (recibieron dos aplicaciones de vacuna).

b) Indicar cual es el top 5 de provincias que han inmunizado de forma completa a mayor cantidad de ciudadanos (es decir realizando dos aplicaciones de la vacuna) en el periodo temporal de diciembre de 2020 a marzo (inclusive) de 2021.



In [1]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import random as rd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [13]:
# generamos valores aleatorio a utilizar en la generacion de los data frames
id_ciudadano = np.random.randint(1, high = 10, size = 20)
id_centro_vacunacion = rd.sample(range(1,21),20)
id_categoria = rd.sample(range(1,21),20)
id_provincia = np.random.randint(1, high = 5, size = 20)


In [11]:
# armamos el df de centros de vacunacion

centros_data = {
        'id_centro_vacunacion': id_centro_vacunacion,
        'nombre': id_centro_vacunacion,
        'direccion': id_centro_vacunacion,
        'codigo_postal': id_centro_vacunacion,
        'partido': id_centro_vacunacion,
        'provincia': id_provincia, 
}

df_centros = pd.DataFrame(centros_data, columns = centros_data.keys())

In [12]:
# armamos el data frame de datos de productos
categoria_data = {
        'id_categoria': id_categoria,
        'nombre_categoria': id_categoria,
        'prioridad': np.random.randint(1, high=5, size=20),
}

df_categoria = pd.DataFrame(categoria_data, columns = categoria_data.keys())

In [13]:
aplicaciones_data = {
        'id_ciudadano': id_ciudadano,
        'fecha': pd.date_range(start = '2020-11-01', end='2021-04-01', periods=20),
        'id_categoria': id_categoria,
        'id_centro_vacunacion': id_centro_vacunacion,
        'id_unico_dosis_vacuna': rd.sample(range(1,199), 20),
        'id_tipo_vacuna': np.random.randint(1,high=3, size=20), 
}

df_aplicaciones = pd.DataFrame(aplicaciones_data, columns = aplicaciones_data.keys())

In [14]:
df_centros.head()

,id_centro_vacunacion,nombre,direccion,codigo_postal,partido,provincia
0,10,10,10,10,10,2
1,9,9,9,9,9,3
2,12,12,12,12,12,2
3,5,5,5,5,5,2
4,20,20,20,20,20,2


In [15]:
df_categoria.head()

,id_categoria,nombre_categoria,prioridad
0,3,3,2
1,18,18,4
2,2,2,1
3,8,8,4
4,16,16,1


In [16]:
df_aplicaciones.head()

,id_ciudadano,fecha,id_categoria,id_centro_vacunacion,id_unico_dosis_vacuna,id_tipo_vacuna
0,5,2020-11-01 00:00:00.000000000,3,10,22,2
1,3,2020-11-08 22:44:12.631578947,18,9,119,2
2,5,2020-11-16 21:28:25.263157894,2,12,91,2
3,9,2020-11-24 20:12:37.894736842,8,5,168,1
4,6,2020-12-02 18:56:50.526315789,16,20,1,1


a) Indicar el porcentaje de ciudadanos vacunados en prioridad 1 que tienen inmunidad completa (recibieron dos aplicaciones de vacuna).

In [20]:
categoria_prioritarias = df_categoria.loc[df_categoria['prioridad']==1,['id_categoria']]

In [21]:
categoria_prioritarias

,id_categoria
2,2
4,16
5,5
7,14
9,13
10,7
15,9


In [23]:
aplicaciones_por_ciudadano = df_aplicaciones.merge(categoria_prioritarias, how='inner')

In [24]:
aplicaciones_por_ciudadano

,id_ciudadano,fecha,id_categoria,id_centro_vacunacion,id_unico_dosis_vacuna,id_tipo_vacuna
0,5,2020-11-16 21:28:25.263157894,2,12,91,2
1,6,2020-12-02 18:56:50.526315789,16,20,1,1
2,5,2020-12-10 17:41:03.157894737,5,6,84,2
3,5,2020-12-26 15:09:28.421052632,14,4,15,1
4,5,2021-01-11 12:37:53.684210526,13,16,62,1
5,1,2021-01-19 11:22:06.315789474,7,3,42,1
6,4,2021-02-28 05:03:09.473684210,9,11,197,2


In [28]:
aplicaciones_por_ciudadano = aplicaciones_por_ciudadano.loc[:,['id_ciudadano','id_categoria']].groupby('id_ciudadano').count().reset_index().rename(columns={'id_categoria':'total_aplicaciones'})

In [29]:
aplicaciones_por_ciudadano

,id_ciudadano,total_aplicaciones
0,1,1
1,4,1
2,5,4
3,6,1


In [36]:
float((len(aplicaciones_por_ciudadano[aplicaciones_por_ciudadano['total_aplicaciones']>1])*100)/len(aplicaciones_por_ciudadano))

25.0

b) Indicar cual es el top 5 de provincias que han inmunizado de forma completa a mayor cantidad de ciudadanos (es decir realizando dos aplicaciones de la vacuna) en el periodo temporal de diciembre de 2020 a marzo (inclusive) de 2021.